In [1]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
data = pd.read_csv("../DATA/vgsales.csv")

In [3]:
data.head(10)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
columns_cat = ['Platform', 'Genre', 'Publisher']
distinct_value = []

for cat_var in columns_cat:
    n = data[cat_var].nunique()
    distinct_value.append(n)

pd.DataFrame(distinct_value).transpose().set_axis(labels = columns_cat, axis = 1)

,Platform,Genre,Publisher
0,31,12,578


## Initial Analysis

In [6]:
data.filter(items=['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']).describe()

,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000


We can see that, overall, North America is the region with most sales, from 1980 to 2020. 

In [7]:
data_long = pd.melt(
    data, 
    id_vars=['Year', 'Rank', 'Name', 'Platform', 'Genre', 'Publisher'],
    value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
    var_name='Place', value_name='Sales'
)

data_overall_sales = data_long.groupby(['Year', 'Place'])['Sales'].sum().reset_index()

In [8]:
plot_line = px.line(data_overall_sales, x='Year', y='Sales', line_group='Place', color='Place',title='Sales Over Year')
plot_line

We can observe that the sales gowth began from 1995, reaching its peak in 2008, where it started to fall.

In [9]:
sales_per_genre = data_long.groupby(['Year', 'Genre'])['Sales'].sum().reset_index()
px.line(sales_per_genre, x='Year', y='Sales', line_group='Genre', color='Genre')

In [10]:
sales_2005 = data_long.loc[(data_long['Year']>=2005)&(data_long['Genre']=='Sports')].sort_values('Year')\
    .groupby(['Year', 'Platform'])['Sales'].sum().reset_index()

px.line(sales_2005, x='Year', y='Sales', line_group='Platform', color='Platform', title='Sales - 2005 - Sports')

In [39]:
sales_wii = data_long.loc[(data_long['Platform']=='Wii')&(data_long['Genre']=='Sports')]\
    .groupby(['Year', 'Name'])['Sales'].sum().reset_index()

sales_wii

,Year,Name,Sales
0,2006.0,Madden NFL 07,1.00
1,2006.0,Rapala Tournament Fishing!,1.11
2,2006.0,Super Swing Golf,0.46
3,2006.0,Wii Sports,165.48
4,2007.0,AMF Bowling Pinbusters!,1.83
...,...,...,...
251,2012.0,NBA 2K13,0.90
252,2012.0,Pro Evolution Soccer 2013,0.19
253,2013.0,FIFA 14,0.86
254,2013.0,Turbo: Super Stunt Squad,0.04


In [47]:
sales_wii = data_long.loc[(data_long['Platform']=='Wii')&(data_long['Genre']=='Sports')]\
    .groupby(['Year', 'Name'])['Sales'].sum().reset_index()

custom_palette = px.colors.qualitative.Set1

px.bar(sales_wii, x='Year', y='Sales', color='Name', title='Wii - Sports Sales')

We can see that from the Wii Platform started pretty well, with 168.05 million sales in its 1st year, given that it was launched in the end of 2006. However, in the next 2 year, 2007 and 2008, it only decreased its sales, lead by a record of sales in the next year, 2009. 